In [ ]:
%load_ext autoreload
%autoreload 2

import logging
from pathlib import Path

from dotenv import dotenv_values

from generator_drainage_units import run_generator_order_levels
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
logging.basicConfig(level=logging.INFO)

In [ ]:
# Define case and base_dir

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
waterschap = "Waterschap Vallei & Veluwe"
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"

# Define waterboard specific variables
range_orde_code_min = 721
range_orde_code_max = 760

In [ ]:
case_path = Path(base_dir, case_name)
order = run_generator_order_levels(
    path=case_path,
    waterschap=waterschap,
    range_orde_code_min=range_orde_code_min,
    range_orde_code_max=range_orde_code_max,
    create_html_map=True,
)

In [ ]:
hydroobjects = gpd.read_file(Path(case_path, '1_tussenresultaat', 'hydroobjecten_processed.gpkg'))

In [ ]:
hydroobjects = hydroobjects.rename(columns={"CODE": "code"})

In [ ]:
from generator_drainage_units.utils.create_graph import create_graph_from_edges
from generator_drainage_units.utils.general_functions import define_list_upstream_downstream_edges_ids, calculate_angles_of_edges_at_nodes

nodes, edges, G = create_graph_from_edges(hydroobjects)
nodes = define_list_upstream_downstream_edges_ids(node_ids=nodes.nodeID.values, nodes=nodes, edges=edges)
nodes = calculate_angles_of_edges_at_nodes(nodes, edges)
nodes

In [ ]:
#hydroobjects = order.hydroobjecten[["CODE", 'NAAM', 'geometry']].copy()
hydroobjects['orde_nr'] = 0

outflow_nodes = order.outflow_nodes_all.copy()
outflow_nodes = outflow_nodes.rename(columns={'hydroobject_code':'hydroobject_code_in'})
outflow_nodes['hydroobject_code_out'] = None
outflow_nodes["orde_nr"] = 1

for orde_nr in range(1,6):
    print(orde_nr)
    outflow_nodes_copy = outflow_nodes[outflow_nodes['orde_nr'] == orde_nr].copy()
    outflow_nodes_copy['geometry'] = outflow_nodes_copy.buffer(0.0001).to_crs(28992)
    end_point_hydroobjects = hydroobjects.copy()
    end_point_hydroobjects['geometry'] = end_point_hydroobjects['geometry'].apply(lambda geom: Point(geom.coords[-1]))
    end_point_hydroobjects = end_point_hydroobjects.set_crs(28992)
    current_hydroobjects = gpd.sjoin(
        outflow_nodes_copy,
        end_point_hydroobjects[['CODE', 'geometry']],
        how="left",
        predicate="intersects",
    )
    current_codes = list(current_hydroobjects['CODE'].values)
    
    while current_codes:
        start_points = hydroobjects[hydroobjects['CODE'].isin(current_codes)]['geometry'].apply(lambda x: x.coords[0]).tolist()
        # Find hydroobjects in gdf_hydroobjects whose endpoints match the start points of the current hydroobjects
        next_hydroobjects = hydroobjects[hydroobjects['geometry'].apply(lambda x: x.coords[-1] in start_points)].copy()
        # Add a column 'next_hydro' to store the hydroobject it ends at
        next_hydroobjects['next_hydro'] = next_hydroobjects['geometry'].apply(
            lambda x: [code for code, geo in zip(hydroobjects['CODE'], hydroobjects['geometry']) if geo.coords[0] == x.coords[-1]]
        )
        hydroobjects.loc[hydroobjects['CODE'].isin(current_codes), 'orde_nr'] = orde_nr + 1
        next_codes = next_hydroobjects['CODE'].tolist()

        # Identify duplicated 'next_hydro' values
        multiple_next_hydroobjects = next_hydroobjects[next_hydroobjects.duplicated('next_hydro', keep=False)]
        single_next_hydroobjects = next_hydroobjects[~next_hydroobjects.duplicated('next_hydro', keep=False)]
        display(multiple_next_hydroobjects)

        # Create a new outflow_nodes based on multiple_next_hydro
        for next_hydro in multiple_next_hydroobjects['next_hydro'].explode().unique():
            # Get hydroobjects with the same next_hydro
            hydroobjects_with_same_next_hydro = multiple_next_hydroobjects[multiple_next_hydroobjects['next_hydro'].explode() == next_hydro]
            
            # Aggregate the hydroobject codes
            # hydroobjects_codes = ','.join(hydroobjects_with_same_next_hydro['CODE'].astype(str))
            hydroobjects_codes = list(hydroobjects_with_same_next_hydro['CODE'].astype(str).values)
            
            # Create a new outflow_node
            new_outflow_node = {
                'geometry': Point(hydroobjects_with_same_next_hydro.iloc[0]['geometry'].coords[-1]),
                'hydroobject_code_out': next_hydro,
                'hydroobject_code_in': hydroobjects_codes,
                'orde_nr': int(orde_nr + 1)
            }
            new_outflow_node = gpd.GeoDataFrame(
                pd.DataFrame([new_outflow_node]), 
                geometry='geometry', 
                crs=outflow_nodes.crs
            )
            # Append the new outflow_node to outflow_nodes
            outflow_nodes = pd.concat([outflow_nodes, new_outflow_node], ignore_index=True)

        current_codes = single_next_hydroobjects['CODE'].tolist()
        break

In [ ]:
next_hydroobjects

In [ ]:
duplicated_hydro

In [ ]:
hydroobjects.to_file(Path(case_path, '1_tussenresultaat', 'test.gpkg'))

In [ ]:
outflow_nodes

In [ ]:
outflow_nodes

In [ ]:
hydroobjects[hydroobjects['orde_nr'] == 2]

In [ ]:
current_codes

In [ ]:
outflow_nodes.rename(columns={'hydroobject_code':'hydroobject_code_in'})

In [ ]:
outflow_nodes["orde_nr"] = 1

In [ ]:
order.hydroobjecten["UITSTROOM_ANGLE"]

In [ ]:
order.outflow_nodes_all

In [ ]:
order.generate_folium_map()

In [ ]:
order.outflow_nodes_all.to_file(Path(order.path, "inflow_outflow_points.gpkg"))